In [2]:
from google.colab import files
uploaded = files.upload()

Saving task1_cleaned.zip to task1_cleaned.zip


In [3]:
import pandas as pd
import zipfile

zip_file = "task1_cleaned.zip"

with zipfile.ZipFile(zip_file) as z:
    with z.open("task1_cleaned.csv") as f:
        df = pd.read_csv(f)

df.head()


/tmp/ipython-input-2233719564.py:8: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,LossRatio
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,NaN


In [6]:
# Claim occurred? 1 = yes, 0 = no
df['ClaimOccurred'] = (df['TotalClaims'] > 0).astype(int)

# Loss ratio
df['LossRatio'] = df['TotalClaims'] / df['TotalPremium']


In [7]:
from statsmodels.stats.proportion import proportions_ztest

# Claim occurred? 1 = yes, 0 = no
df['ClaimOccurred'] = (df['TotalClaims'] > 0).astype(int)

# Split by Gender
male = df[df['Gender'] == 'M']['ClaimOccurred']
female = df[df['Gender'] == 'F']['ClaimOccurred']

# Number of successes and total observations
count = [male.sum(), female.sum()]
nobs = [len(male), len(female)]

# Z-test
stat, pval = proportions_ztest(count, nobs)

print("Gender Claim Frequency Test:")
print(f"Z-statistic: {stat:.4f}, p-value: {pval:.4f}")

if pval < 0.05:
    print("Reject H0: Significant difference in claim frequency between genders")
else:
    print("Fail to reject H0: No significant difference")


Gender Claim Frequency Test:
Z-statistic: nan, p-value: nan
Fail to reject H0: No significant difference


/usr/local/lib/python3.12/dist-packages/statsmodels/stats/proportion.py:1004: RuntimeWarning: invalid value encountered in divide
  prop = count * 1. / nobs
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/proportion.py:1018: RuntimeWarning: invalid value encountered in scalar divide
  p_pooled = np.sum(count) * 1. / np.sum(nobs)
/usr/local/lib/python3.12/dist-packages/statsmodels/stats/proportion.py:1020: RuntimeWarning: divide by zero encountered in divide
  nobs_fact = np.sum(1. / nobs)


In [8]:
from statsmodels.stats.proportion import proportions_ztest

male = df[df['Gender'] == 'M']['ClaimOccurred']
female = df[df['Gender'] == 'F']['ClaimOccurred']

count = [male.sum(), female.sum()]
nobs = [len(male), len(female)]

stat, pval = proportions_ztest(count, nobs)

print("Gender Claim Frequency Test:")
print(f"Z-statistic: {stat:.4f}, p-value: {pval:.4f}")
if pval < 0.05:
    print("Reject H0: Significant difference in claim frequency between genders")
else:
    print("Fail to reject H0: No significant difference")


Gender Claim Frequency Test:
Z-statistic: nan, p-value: nan
Fail to reject H0: No significant difference


In [9]:
from scipy.stats import chi2_contingency

# Example: ClaimOccurred vs Province
contingency = pd.crosstab(df['Province'], df['ClaimOccurred'])
chi2, p, dof, ex = chi2_contingency(contingency)

print("Province Claim Frequency Test:")
print(f"Chi2: {chi2:.2f}, p-value: {p:.4f}")

if p < 0.05:
    print("Reject H0: Risk differs across provinces")
else:
    print("Fail to reject H0: No significant difference across provinces")


Province Claim Frequency Test:
Chi2: 104.19, p-value: 0.0000
Reject H0: Risk differs across provinces
